<h1 align= "center"> Week 2 </h1>
<h3> Aim: </h3>

1. Download BACE1 protein from pdb (7MYI -- BACE1 closed-flap -- newest gold-standard model -- res = 1.45 Å)
2. Clean the 7MYI BACE1 protein model using OpenBabel/rdkit and saving the cleaned model as well as the bound legand separately.
3. Prepare receptor (rigid PDBQT) and ligand (flexible PDBQT) files compatible with AutoDock Vina / QuickVina 2.
4. Calculate heavy-atom symmetry-corrected RMSD between the top-ranked docked pose and the experimental crystal pose.
5. Visualize the receptor, reference ligand, and best docked pose in 3D (NGLView) to confirm overlay quality.

 

<h3> Method: </h3>

In [5]:
import sys
import subprocess
import tempfile
import os
import nglview as nv
import urllib.request
import numpy as np

from pathlib import Path
from spyrmsd import io, rmsd
from openbabel import pybel
from meeko import MoleculePreparation, PDBQTWriterLegacy
from rdkit import Chem
from rdkit.Chem import rdMolAlign, AllChem
from IPython.display import display

Downloading the most up-to-date BACE1 enzyme:

In [3]:
raw_pdb = "../data/7MYI_raw.pdb"
if not os.path.exists(raw_pdb):
    print("Downloading 7MYI...")
    urllib.request.urlretrieve("https://files.rcsb.org/download/7MYI.pdb", raw_pdb)

In [4]:
mol = Chem.MolFromPDBFile(raw_pdb, removeHs=False, sanitize=False)

# Filter: chain A only, no waters
keep_atoms = []
for atom in mol.GetAtoms():
    res_info = atom.GetPDBResidueInfo()
    if res_info:
        res_name = res_info.GetResidueName().strip()
        chain_id = res_info.GetChainId()
        if res_name != 'HOH' and chain_id == 'A':
            keep_atoms.append(atom.GetIdx())

# Create new mol with kept atoms
edited = Chem.EditableMol(mol)
for i in sorted(range(mol.GetNumAtoms()), reverse=True):
    if i not in keep_atoms:
        edited.RemoveAtom(i)
new_mol = edited.GetMol()

# Separate protein (standard amino acid "aa") and ligand (non-aa)
standard_aa = {"ALA","ARG","ASN","ASP","CYS","GLN","GLU","GLY","HIS","ILE",
               "LEU","LYS","MET","PHE","PRO","SER","THR","TRP","TYR","VAL"}

protein_atoms = []
ligand_atoms = []
for atom in new_mol.GetAtoms():
    res_info = atom.GetPDBResidueInfo()
    if res_info:
        res_name = res_info.GetResidueName().strip()
        if res_name in standard_aa:
            protein_atoms.append(atom.GetIdx())
        else:
            ligand_atoms.append(atom.GetIdx())

# Protein molecule
protein_ed = Chem.EditableMol(new_mol)
for i in sorted(range(new_mol.GetNumAtoms()), reverse=True):
    if i not in protein_atoms:
        protein_ed.RemoveAtom(i)
protein = protein_ed.GetMol()
protein = Chem.AddHs(protein, addCoords=True)
Chem.MolToPDBFile(protein, '../data/bace1_protein_H.pdb')

# Ligand molecule
ligand_ed = Chem.EditableMol(new_mol)
for i in sorted(range(new_mol.GetNumAtoms()), reverse=True):
    if i not in ligand_atoms:
        ligand_ed.RemoveAtom(i)
ligand = ligand_ed.GetMol()
w = Chem.SDWriter('../data/bace1_ligand.sdf')
w.write(ligand)
w.close()

print("Cleaning complete:")
print(f"  Protein atoms: {len(protein_atoms)}")
print(f"  Ligand atoms: {len(ligand_atoms)}")
print("  → bace1_protein_H.pdb")
print("  → bace1_ligand.sdf")

Cleaning complete:
  Protein atoms: 3066
  Ligand atoms: 21
  → bace1_protein_H.pdb
  → bace1_ligand.sdf


Preparing the BACE1 protein and ligand files with meeko to be usable by vina

In [ ]:
# ------------------- PROTEIN (RECEPTOR) -------------------
print("--- Starting Protein Preparation ---")

# Use OpenBabel (pybel) for the receptor. 
# It is fast and handles the rigid structure correctly without "hanging".
protein_file = "../data/bace1_protein_H.pdb"
protein_output = "../data/protein.pdbqt"

try:
    # 1. Read the PDB file
    protein1 = next(pybel.readfile("pdb", protein_file))
    
    # 2. Add Hydrogens (crucial for docking)
    protein1.addh()
    
    # 3. Write directly to PDBQT 
    # OpenBabel automatically handles the atom typing and conversion for rigid receptors.
    protein1.write("pdbqt", protein_output, overwrite=True)
    
    print(f"Protein written successfully to: {protein_output}")

except Exception as e:
    print(f"Error preparing protein: {e}")


# ------------------- LIGAND PREPARATION -------------------
print("--- Starting Ligand Preparation ---")

input_path = "../data/bace1_ligand.sdf"
output_path = "../data/ligand.pdbqt"

try:
    # 1. Read the SDF file
    ligand = next(pybel.readfile("sdf", input_path))
    
    # 2. Add Hydrogens 
    ligand.addh()
    
    # 3. Calculate Partial Charges (Gasteiger) & Write PDBQT
    ligand.write("pdbqt", output_path, overwrite=True)
    
    print(f"Success! Ligand written to: {output_path}")
    
    # 4. Verification
    # Check if the file is not empty
    if os.path.getsize(output_path) > 0:
        with open(output_path, 'r') as f:
            lines = f.readlines()
            print(f"File created with {len(lines)} lines.")
            # Quick check for Torsion Tree keywords
            if any("ROOT" in line for line in lines):
                print("Verification: Torsion Tree (ROOT/BRANCH) detected.")
            else:
                print("Warning: Torsion keywords not found (Ligand might be treated as rigid).")
    else:
        print("Error: The output file is empty.")

except Exception as e:
    print(f"An error occurred: {e}")

--- Starting Protein Preparation ---
Protein written successfully to: ../data/protein.pdbqt
--- Starting Ligand Preparation ---
Success! Ligand written to: ../data/ligand.pdbqt
File created with 43 lines.
Verification: Torsion Tree (ROOT/BRANCH) detected.


In [ ]:
input_file = "../data/protein.pdbqt" 
# The new, rigid file ready for Vina
output_file = "../data/protein_rigid.pdbqt" 

print(f"--- Preparing Rigid Receptor: {output_file} ---")

if not os.path.exists(input_file):
    print(f"Error: Input file not found at {input_file}")
else:
    # 1. Read the original content
    with open(input_file, 'r') as infile:
        lines = infile.readlines()
    
    rigid_lines = []
    
    # 2. Filter out all flexibility-defining keywords
    for line in lines:
        # Skip all REMARK lines (contains torsion status, name, etc.)
        if line.startswith('REMARK'):
            continue
        # Skip all BRANCH/ENDBRANCH/ROOT/ENDROOT lines (defines the torsion tree)
        if line.startswith('BRANCH') or line.startswith('ENDBRANCH') or line.startswith('ROOT') or line.startswith('ENDROOT'):
            continue
        
        # Keep only ATOM and HETATM records (coordinates, charges, atom types)
        if line.startswith('ATOM') or line.startswith('HETATM'):
            rigid_lines.append(line)
    
    # Add an 'END' record, which is standard PDBQT/PDB practice
    rigid_lines.append('END\n')

    # 3. Write the cleaned content to the new file
    with open(output_file, 'w') as outfile:
        outfile.writelines(rigid_lines)
    
    print(f"Original lines processed: {len(lines)}")
    print(f"Rigid lines saved: {len(rigid_lines)}")
    print(f"Rigid receptor successfully created at: {output_file}")

--- Preparing Rigid Receptor: ../data/protein_rigid.pdbqt ---
Original lines processed: 7221
Rigid lines saved: 3735
✅ Rigid receptor successfully created at: ../data/protein_rigid.pdbqt


Vina validation docking:

1. Automatically calculate the precise geometric centroid of the co-crystallized ligand to define the optimal docking box

In [7]:
# Load the co-crystallized ligand (the one you extracted)
lig = Chem.MolFromMolFile("../data/bace1_ligand.sdf", removeHs=False)

# Get 3D coordinates
conf = lig.GetConformer()
coords = np.array([conf.GetAtomPosition(i) for i in range(lig.GetNumAtoms())])

# Calculate geometric centroid
centroid = np.mean(coords, axis=0)

print("7MYI co-crystallized ligand centroid (Å):")
print(f"   x = {centroid[0]:6.3f}")
print(f"   y = {centroid[1]:6.3f}")
print(f"   z = {centroid[2]:6.3f}")

7MYI co-crystallized ligand centroid (Å):
   x = 20.555
   y = 55.427
   z = 87.801


2. AutoDock Vina

In [8]:
!vina --receptor "../data/protein_rigid.pdbqt" --ligand "../data/ligand.pdbqt" --center_x 20.6 --center_y 55.4 --center_z 87.8 --size_x 25 --size_y 25 --size_z 25 --cpu 8 --exhaustiveness 128 --num_modes 10 --out "../results/7MYI_redocked.pdbqt"

############################################################################
# If you used Quick Vina 2 in your work, please cite:                      #
#                                                                          #
# Amr Alhossary, Stephanus Daniel Handoko, Yuguang Mu, and Chee-Keong Kwoh,#
# Fast, Accurate, and Reliable Molecular Docking with QuickVina 2,         #
# Bioinformatics (2015), doi: 10.1093/bioinformatics/btv082                #
#                                                                          #
# You are also encouraged to cite Quick Vina 1:                            #
# Stephanus Daniel Handoko, Xuchang Ouyang, Chinh Tran To Su, Chee Keong   #
# Kwoh, Yew Soon Ong,                                                      #
# QuickVina: Accelerating AutoDock Vina Using Gradient-Based Heuristics for#
# Global Optimization,                                                     #
# IEEE/ACM Transactions on Computational Biology and Bioinformatics,vol.9, #

RMSD calculation:

In [ ]:
# Heavy-atom RMSD (hydrogens stripped) – standard metric in docking validation
ref = io.loadmol("../data/ligand.pdbqt")
probe = io.loadallmols("../results/7MYI_redocked.pdbqt")  

ref.strip()
probe[0].strip()  

rmsd_value = rmsd.symmrmsd(
    ref.coordinates,
    probe[0].coordinates,
    ref.atomicnums,
    probe[0].atomicnums,
    ref.adjacency_matrix,
    probe[0].adjacency_matrix
)

print(f"RMSD: {rmsd_value:.3f} Å")  

RMSD: 1.043 Å


<h3> Results and Visualization: </h3>

In [31]:
# --- 1. Define File Paths ---
RECEPTOR_PDBQT = "../data/protein_rigid.pdbqt"
REF_LIGAND_PDBQT = "../data/ligand.pdbqt"
DOCKED_LIGAND_PDBQT = "../results/7MYI_redocked.pdbqt"

# Temporary PDB files for NGLView
TEMP_RECEPTOR_PDB = "ngl_temp_receptor.pdb"
TEMP_REF_LIGAND_PDB = "ngl_temp_ref_ligand.pdb"
TEMP_DOCKED_LIGAND_PDB = "ngl_temp_docked_ligand.pdb"

# --- 2. Conversion Function (Using Pybel) ---
def convert_pdbqt_to_pdb(input_path, output_path, is_multi=False):
    """Converts PDBQT to PDB using Pybel, getting only the first pose if multi-pose file."""
    if is_multi:
        # Read all poses (only need the first one, which is the best Vina result)
        mols = list(pybel.readfile("pdbqt", input_path))
        if not mols: raise ValueError("No molecules found in multi-pose file.")
        mol_to_write = mols[0] 
    else:
        # Get only the single molecule
        mol_to_write = next(pybel.readfile("pdbqt", input_path))
        
    mol_to_write.write("pdb", output_path, overwrite=True)

def color_ligand_explicitly(view, component_index, color):
    """
    Creates an explicit coloring scheme for all atoms in a given component.
    This is a more robust way to ensure the color is applied.
    """
    atom_count = view.get_structure_string(component_index).count('\nATOM')
    color_list = [color] * atom_count
    view.add_representation('ball+stick', selection='all', component=component_index, color_scheme='atomindex', color_list=color_list)

# --- 3. Preparation and Loading ---

print("Preparing files for NGLView using temporary PDB files...")

try:
    # A. Convert all PDBQT files to temporary PDB format
    convert_pdbqt_to_pdb(RECEPTOR_PDBQT, TEMP_RECEPTOR_PDB)
    convert_pdbqt_to_pdb(REF_LIGAND_PDBQT, TEMP_REF_LIGAND_PDB)
    # first pose (best affinity) from the Vina output only
    convert_pdbqt_to_pdb(DOCKED_LIGAND_PDBQT, TEMP_DOCKED_LIGAND_PDB, is_multi=True)

    # --- 4. Create and Configure NGLView ---

    # Start the view with the receptor
    view = nv.show_structure_file(TEMP_RECEPTOR_PDB)
    view.clear_representations()
    view.add_representation('cartoon', selection='protein', color="#15610B", component=0)

    # --- 5. Add Ligands and Apply Explicit Coloring ---

    # Reference Ligand (will become Component 1)
    view.add_component(TEMP_REF_LIGAND_PDB)
    # Docked Ligand (will become Component 2)
    view.add_component(TEMP_DOCKED_LIGAND_PDB)

    view.add_representation('ball+stick', selection='all', component=1, name='Reference Pose', color='blue')
    view.add_representation('ball+stick', selection='all', component=2, name='Predicted Pose (Vina)', color='red')
    
    view.center()
    display(view)

    print("\nVisualization ready. Check the overlay:")
    print(f"Blue Ball and stick = Reference Pose (Component {ref_comp.id})")
    print(f"Red Ball and stick = Predicted Pose (Component {docked_comp.id})")

except Exception as e:
    print(f"An error occurred during visualization. Ensure all PDBQT files exist. Error: {e}")

Preparing files for NGLView using temporary PDB files...


NGLWidget()


Visualization ready. Check the overlay:
Blue Ball and stick = Reference Pose (Component e0d3ccf1-2b0d-4ad2-9142-0f719ca807eb)
An error occurred during visualization. Ensure all PDBQT files exist. Error: name 'docked_comp' is not defined


![alt text](../figures/ligand-docking-validation2.PNG) ![alt text](../figures/ligand-docking-validation1.PNG)

- Successfully cleaned 7MYI → protein with 3,066 heavy atoms + explicit polar hydrogens
- Co-crystallized ligand correctly extracted (21 heavy atoms)
- Receptor and ligand PDBQT files generated without errors; torsion tree properly detected for the ligand
- Docking box centered on ligand centroid: x = 20.6 Å, y = 55.4 Å, z = 87.8 Å
- QuickVina 2 redocking completed in approximately 9 minutes (8 CPU threads, exhaustiveness 128)
- Best docked pose affinity: –8.3 kcal/mol (identical to previous run, confirming reproducibility)
- Heavy-atom symmetry-corrected RMSD = 1.043 Å (well below the widely accepted <2.0 Å threshold for successful redocking)
- 3D visualization confirms near-perfect overlap between crystal pose (blue) and predicted pose (red)